In [1]:
import pandas as pd
# data = pd.read_csv("freq-59g-mwe62m.csv.gz", nrows=100, compression='gzip')
from tqdm import tqdm
import gzip
with gzip.open("freq-59g-mwe62m.csv.gz", 'rb') as f:
    header = f.readlines()

In [134]:
data = pd.read_csv("data/adaptive_rag_natural_questions/test.csv")

In [127]:
len(header)

40287725

In [4]:
dict_collocations = dict()

for j in tqdm(range(len(header))):
    try:
        collocation = str(header[j]).split('\\t')[0].split("b'")[1]
        amount = int(str(header[j]).split('\\t')[1].split("\\n'")[0])
        dict_collocations[collocation] = amount
    except:
        continue

100%|██████████| 40287725/40287725 [01:01<00:00, 655154.21it/s]


In [13]:
len(header)-len(dict_collocations)

2076359

In [135]:
for i in tqdm(range(len(data))):
    if pd.notna(data.loc[i, 'bela_base_ents_lbls']):
        ents = data.loc[i, 'bela_base_ents_lbls'].split(", ")
        cur_pops = []
        for j in range(len(ents)):
            cur_ent = ents[j].replace("'s", "").replace("?", "").replace(".", "").replace(",", "")
            if cur_ent in dict_collocations:
                cur_pops.append(dict_collocations[cur_ent])
        data.loc[i, 'frequency_exact'] = str(cur_pops)

100%|██████████| 500/500 [00:00<00:00, 7663.07it/s]


In [136]:
def all_collocations(question):
    
    words = question.split(" ")
    all_words = words.copy()
    for i in range(len(words)-1):
        all_words.append(' '.join(words[i:2+i]))
    for i in range(len(words)-2):
        all_words.append(' '.join(words[i:3+i]))
    for i in range(len(words)-3):
        all_words.append(' '.join(words[i:4+i]))
    for i in range(len(words)-4):
        all_words.append(' '.join(words[i:5+i]))
    all_words = [elem.replace("'s", "").replace("?", "").replace(".", "").replace(",", "") for elem in all_words]

    return all_words

In [137]:
for i in tqdm(range(len(data))):
    
    cur_ques  = data.loc[i, 'question_text']
    all_collocs = all_collocations(cur_ques)
    pops = []
    for j in range(len(all_collocs)):
        cur_key = all_collocs[j]
        if cur_key in dict_collocations:
            pops.append(dict_collocations[cur_key])
    data.loc[i, 'frequency_inexact_all'] = str(pops)

100%|██████████| 500/500 [00:00<00:00, 5622.93it/s]


In [139]:
data.to_csv("data/adaptive_rag_natural_questions/test.csv", index = False)